# Práctica 1 - NNs

### Natalia Martínez García, Lucía Vega Navarrete
### Grupo: AP.11.06

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from keras import layers, models, regularizers, metrics
import numpy as np
import matplotlib.pyplot as plt

import random
random.seed(123)

En esta práctica usamos el dataset STL-10, que contiene imágenes de 96×96 en color y 10 clases (avión, pájaro, coche, gato, ciervo, perro, caballo, mono, barco, camión). El objetivo es entrenar una red neuronal totalmente conectada para clasificar estas imágenes.
Primero cargamos y preprocesamos los datos: normalizamos las imágenes, convertimos las etiquetas a one-hot y aplanamos cada imagen en un vector. Después entrenamos varios modelos cambiando la regularización para intentar mejorar la generalización y evitar sobreajuste.

### 1. Carga del dataset.

In [15]:
# Cargamos el dataset STL-10 ya dividido en entrenamiento y test.
# as_supervised=True hace que cada elemento tenga forma (imagen, etiqueta)
# with_info=True también devuelve info extra del dataset (número de clases, tamaño de imagen, etc.)

(train, test), info_ds = tfds.load(
    'stl10',
    split=['train', 'test'],
    # shuffle_files=True, # PARA QUE VENGAN DESORDENADOS
    as_supervised=True,  # Devuelve tuplas (imagen, etiqueta)
    with_info=True
)

num_clases = info_ds.features['label'].num_classes
nombres_clases = info_ds.features['label'].names
tamano_imagen = info_ds.features['image'].shape
dimension_entrada = np.prod(tamano_imagen)

print("\n" + "="*50)
print("INFORMACIÓN DEL DATASET")
print("="*50)

print(f"NOMBRE: {info_ds.name}")
print(f"\nIMÁGENES:")
print(f" - Dimensiones: {tamano_imagen}")
print(f" - Tipo: {info_ds.features['image'].numpy_dtype}")
print(f" - Longitud aplanada: {dimension_entrada}")


print(f"\nETIQUETAS:")
print(f" - Número de clases: {num_clases}")
print(f" - Clases: {', '.join(nombres_clases)}")

print(f"\nSPLITS:")
print(f" - Train: {info_ds.splits['train'].num_examples:,} imágenes")
print(f" - Test: {info_ds.splits['test'].num_examples:,} imágenes")
print(f" - Unlabelled: {info_ds.splits['unlabelled'].num_examples:,} imágenes (NO LOS USAMOS)")


INFORMACIÓN DEL DATASET
NOMBRE: stl10

IMÁGENES:
 - Dimensiones: (96, 96, 3)
 - Tipo: <class 'numpy.uint8'>
 - Longitud aplanada: 27648

ETIQUETAS:
 - Número de clases: 10
 - Clases: airplane, bird, car, cat, deer, dog, horse, monkey, ship, truck

SPLITS:
 - Train: 5,000 imágenes
 - Test: 8,000 imágenes
 - Unlabelled: 100,000 imágenes (NO LOS USAMOS)


### 2. Preprocesado del dataset

In [4]:
def preprocesado(imagen, etiqueta):
    imagen = tf.cast(imagen, tf.float32) / 255.0  # imagen a float32 y escala [0,1]
    imagen = tf.reshape(imagen, [-1]) # aplanamos
    etiqueta = tf.one_hot(etiqueta, depth = num_clases) # one-hot
    return imagen, etiqueta

def preprocesado_dataset(dataset):
    imagenes = []
    etiquetas = []

    for img, label in dataset:
        imagen, etiqueta = preprocesado(img, label)
        imagenes.append(imagen)
        etiquetas.append(etiqueta)

    return np.array(imagenes), np.array(etiquetas)

train_inputs, train_targets = preprocesado_dataset(train)
test_inputs, test_targets = preprocesado_dataset(test)

Las imágenes vienen en formato uint8 (enteros sin signo), con valores de píxeles  entre 0 y 255. Escalamos los píxeles al rango [0, 1] dividiendo por 255 para que la red converja más rápido y de forma más estable.

Además, las imágenes son matrices de 96×96×3 (alto, ancho, canales RGB), pero una red neuronal densa solo puede trabajar con vectores 1D. Por eso necesitamos aplanar la imagen: el resultado es un vector de longitud 27648. Con esto la red no conoce que un píxel está al lado del otro (pierde la información espacial).

Usamos one-hot encoding cuando tenemos variables categóricas que no tiene sentido ordenar (no hay una categoría “mayor” o “menor” que la otra), como es nuestro caso con las 10 clases del dataset (avión, pájaro, coche, gato, etc.). Con esto convertimos una etiqueta categórica en un vector donde solo la posición correspondiente a esa categoría tiene un 1 y el resto son 0s (por ejemplo, la clase 0 se convierte en [1,0,0,0,0,0,0,0,0,0]). Si dejáramos las etiquetas como números del 0 al 9, podría interpretarse erróneamente que existe una relación de orden entre ellas. AÑADIR ALGO DEL CROSSENTROPY ?? LUEGO CUANDO LO ENTENDAMOS


In [ ]:
# SEPARAR PARTE DE TRAINING PARA VALIDATION

indices_permutation = np.random.permutation(len(train_inputs))
shuffled_inputs = train_inputs[indices_permutation]
shuffled_targets = train_targets[indices_permutation]

num_validation_samples = int(0.2 * len(train_inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]

Para la división del dataset, se mezclan las muestras del conjunto de entrenamiento, de manera que la separación entre entrenamiento y validación sea aleatoria.  

Luego, del conjunto de entrenamiento ya barajado, se toma un 20 % de las muestras para formar el conjunto de validación, mientras que el 80 % restante se mantiene como conjunto de entrenamiento. Este porcentaje es el más habitual en aprendizaje automático, ya que permite evaluar correctamente el modelo sin reducir demasiado la cantidad de datos disponibles para entrenar. El conjunto de validación no participa en el aprendizaje del modelo, sino que se usa durante el entrenamiento para comprobar su progreso y detectar posibles casos de sobreajuste, asegurando que el modelo generalice bien a datos nuevos.

El conjunto de test, por su parte, ya viene definido por defecto en el dataset STL-10, separado del entrenamiento. Este conjunto no se modifica ni se utiliza durante el proceso de entrenamiento o validación, y se reserva exclusivamente para evaluar el rendimiento final sobre nuevos datos que no ha visto antes una vez que ya está completamente entrenado.

Los datos se agrupan en batches de 32 muestras, es decir, el modelo procesa 32 imágenes por iteración antes de actualizar sus pesos. Elegimos este tamaño porque es lo suficientemente pequeño para que el aprendizaje consuma demasiada memoria, pero lo bastante grande para aprovechar la eficiencia del cálculo en paralelo.

### 3. Creación y entrenamiento de modelos

Como queremos entrenar varios modelos y guradar sus historiales y métricas, decidimos hacer una función general de entrenamiento para no repetir código.

In [ ]:
def entrenar(modelo, train, val, test, epochs=15):
    history = modelo.fit(train, validation_data=val, epochs=epochs, verbose=1)

    # Evalúa en test: devuelve [loss, accuracy, precision, recall, f1_score]
    loss, acc, prec, rec, f1 = modelo.evaluate(test, verbose=0)

    # Muestra los resultados de forma ordenada
    print("\nResultados en TEST:")
    print(f"Loss: {loss:.4f}")
    print(f"Accuracy: {acc:.4f}")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-score: {f1:.4f}")

    return history, loss, acc, prec, rec, f1

<p style="color:red; font-weight:bold; line-height:1.5;">
- XQ ESE NÚMERO DE EPOCHS. Ns q criterio se sigue lmao<br>
- XQ ESAS MÉTRICAS
</p>


<p style="color:red; font-weight:bold; line-height:1.5;">
PARA LOS MODELOS: <br>
- Explicar cada cosa del compile<br>
- El num de neuronas en las capas ocultas y la activación <br>
- El dropout <br>
- regularizaciones
</p>

#### 3.1) Modelo 1- Vanilla (sin regularización)

In [ ]:
# Función que construye el modelo básico
def vanilla():
    # Creamos un modelo secuencial, las capas se apilan una tras otra
    modelo = models.Sequential([
        layers.Input(shape=(dimension_entrada,)),  # Capa de entrada con el tamaño del vector aplanado
        layers.Dense(512, activation='relu'), # Primera capa oculta con 512 neuronas y activación ReLU
        layers.Dense(256, activation='relu'), # Segunda capa oculta con 256 neuronas y activación ReLU
        layers.Dense(num_clases, activation='softmax') # Capa de salida con tantas neuronas como clases (10) y activación softmax. softmax devuelve probabilidades para cada clase
    ])

    # optimizador base: Adam (muy estándar hoy en día)
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy', # porcentaje total de aciertos
            metrics.Precision(name='precision'),# mide falsos positivos
            metrics.Recall(name='recall'),  # mide falsos negativos
            metrics.F1Score(name='f1_score', average='macro')  # media F1 por clase
        ]
    )
    return modelo

modelo_base = vanilla() # Creamos una instancia del modelo base llamando a la función
modelo_base.summary() # Mostramos un resumen con el número de capas, parámetros y formas de entrada/salida

In [ ]:
hist_vanilla, loss_vanilla, acc_vanilla, prec_vanilla, rec_vanilla, f1_vanilla = entrenar(
    modelo_base,
    conjunto_entrenamiento,
    conjunto_validacion,
    conjunto_test)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 15s 99ms/step - accuracy: 0.2050 - f1_score: 0.2044 - loss: 4.6240 - precision: 0.2362 - recall: 0.1110 - val_accuracy: 0.3150 - val_f1_score: 0.2655 - val_loss: 2.0319 - val_precision: 0.5526 - val_recall: 0.1260
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 95ms/step - accuracy: 0.2735 - f1_score: 0.2705 - loss: 2.1378 - precision: 0.4540 - recall: 0.1210 - val_accuracy: 0.3160 - val_f1_score: 0.2738 - val_loss: 1.8717 - val_precision: 0.5000 - val_recall: 0.1170
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 92ms/step - accuracy: 0.3142 - f1_score: 0.3118 - loss: 1.9128 - precision: 0.5535 - recall: 0.1138 - val_accuracy: 0.3200 - val_f1_score: 0.2909 - val_loss: 1.9055 - val_precision: 0.6932 - val_recall: 0.1220
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.3128 - f1_score: 0.3096 - loss: 1.8692 - precision: 0.6047 - recall: 0.1170 - val_accuracy: 0.3600 - val_f1_score: 0.3277 - val_loss: 1.7159 - val_precision: 0.6326 - val

#### 3.2) Modelo 2 - Dropout

**Notitas**
- con dropout 0.5: El rendimiento es una shit. Precisión y recall son casi nulos, indicando que el dropout fue demasiado alto (0.5) o aplicado en capas pequeñas

In [ ]:
def dropout():
    modelo = models.Sequential([
        layers.Input(shape=(dimension_entrada,)),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(num_clases, activation='softmax')
    ])

    modelo.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',
    metrics=[
        'accuracy', # porcentaje total de aciertos
        metrics.Precision(name='precision'),# mide falsos positivos
        metrics.Recall(name='recall'),  # mide falsos negativos
        metrics.F1Score(name='f1_score', average='macro')  # media F1 por clase
    ])

    return modelo

modelo_dropout = dropout()
modelo_dropout.summary()

In [ ]:
hist_dropout, loss_dropout, acc_dropout, prec_dropout, rec_dropout, f1_dropout = entrenar(
    modelo_dropout,
    conjunto_entrenamiento,
    conjunto_validacion,
    conjunto_test)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.1410 - f1_score: 0.1267 - loss: 2.1861 - precision: 0.3206 - recall: 0.0105 - val_accuracy: 0.1670 - val_f1_score: 0.0928 - val_loss: 2.0759 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 94ms/step - accuracy: 0.1375 - f1_score: 0.1253 - loss: 2.1684 - precision: 0.3462 - recall: 0.0090 - val_accuracy: 0.1630 - val_f1_score: 0.0798 - val_loss: 2.0354 - val_precision: 0.3333 - val_recall: 0.0010
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 92ms/step - accuracy: 0.1468 - f1_score: 0.1251 - loss: 2.1469 - precision: 0.3656 - recall: 0.0085 - val_accuracy: 0.1850 - val_f1_score: 0.1012 - val_loss: 2.0691 - val_precision: 0.7273 - val_recall: 0.0080
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 12s 93ms/step - accuracy: 0.1500 - f1_score: 0.1121 - loss: 2.1473 - precision: 0.3846 - recall: 0.0100 - val_accuracy: 0.1910 - val_f1_score: 0.0820 - val_loss: 2.0307 - val_precision: 0.00

#### 3.3) Modelo 3 - Regularización L1

In [ ]:
def L1():
    modelo = models.Sequential([
        layers.Input(shape=(dimension_entrada,)),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l1(1e-4)),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l1(1e-4)),
        layers.Dense(num_clases, activation='softmax')
    ])
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy', # porcentaje total de aciertos
            metrics.Precision(name='precision'),# mide falsos positivos
            metrics.Recall(name='recall'),  # mide falsos negativos
            metrics.F1Score(name='f1_score', average='macro')  # media F1 por clase
        ])

    return modelo

modelo_L1 = L1()
modelo_L1.summary()

In [ ]:
hist_L1, loss_L1, acc_L1, prec_L1, rec_L1, f1_L1 = entrenar(
    modelo_L1,
    conjunto_entrenamiento,
    conjunto_validacion,
    conjunto_test)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 123ms/step - accuracy: 0.2855 - f1_score: 0.2828 - loss: 3.5304 - precision: 0.4698 - recall: 0.1110 - val_accuracy: 0.2770 - val_f1_score: 0.2088 - val_loss: 3.4165 - val_precision: 0.5780 - val_recall: 0.1000
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 123ms/step - accuracy: 0.2815 - f1_score: 0.2773 - loss: 3.3984 - precision: 0.4851 - recall: 0.0935 - val_accuracy: 0.2290 - val_f1_score: 0.1848 - val_loss: 3.4605 - val_precision: 0.3267 - val_recall: 0.0980
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 127ms/step - accuracy: 0.2822 - f1_score: 0.2816 - loss: 3.3298 - precision: 0.5035 - recall: 0.1088 - val_accuracy: 0.2530 - val_f1_score: 0.1866 - val_loss: 3.2667 - val_precision: 0.3799 - val_recall: 0.1250
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 17s 126ms/step - accuracy: 0.2955 - f1_score: 0.2940 - loss: 3.1494 - precision: 0.5413 - recall: 0.1130 - val_accuracy: 0.3140 - val_f1_score: 0.2721 - val_loss: 3.1538 - val_precision: 0.5685 -

#### 3.4) Modelo 4 - Regularización L2

In [ ]:
def L2():
    modelo = models.Sequential([
        layers.Input(shape=(dimension_entrada,)),
        layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dense(num_clases, activation='softmax')
    ])
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy', # porcentaje total de aciertos
            metrics.Precision(name='precision'),# mide falsos positivos
            metrics.Recall(name='recall'),  # mide falsos negativos
            metrics.F1Score(name='f1_score', average='macro')  # media F1 por clase
        ])
    return modelo

modelo_L2 = L2()
modelo_L2.summary()

In [ ]:
hist_L2, loss_L2, acc_L2, prec_L2, rec_L2, f1_L2 = entrenar(
    modelo_L2,
    conjunto_entrenamiento,
    conjunto_validacion,
    conjunto_test)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 121ms/step - accuracy: 0.3537 - f1_score: 0.3507 - loss: 1.8618 - precision: 0.6377 - recall: 0.1285 - val_accuracy: 0.3950 - val_f1_score: 0.3641 - val_loss: 1.7650 - val_precision: 0.7268 - val_recall: 0.1410
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 124ms/step - accuracy: 0.3708 - f1_score: 0.3641 - loss: 1.8129 - precision: 0.6444 - recall: 0.1450 - val_accuracy: 0.3870 - val_f1_score: 0.3622 - val_loss: 1.7430 - val_precision: 0.6920 - val_recall: 0.1550
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 122ms/step - accuracy: 0.3898 - f1_score: 0.3808 - loss: 1.7607 - precision: 0.6721 - recall: 0.1645 - val_accuracy: 0.4210 - val_f1_score: 0.3906 - val_loss: 1.6554 - val_precision: 0.6932 - val_recall: 0.1740
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 16s 124ms/step - accuracy: 0.3828 - f1_score: 0.3756 - loss: 1.7465 - precision: 0.6335 - recall: 0.1720 - val_accuracy: 0.3940 - val_f1_score: 0.3631 - val_loss: 1.6988 - val_precision: 0.6827 -

#### 3.5) Modelo 5 - BatchNorm

In [ ]:
def batchnorm():
    modelo = models.Sequential([
        layers.Input(shape=(dimension_entrada,)),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(num_clases, activation='softmax')
    ])
    modelo.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='categorical_crossentropy',
        metrics=[
            'accuracy', # porcentaje total de aciertos
            metrics.Precision(name='precision'),# mide falsos positivos
            metrics.Recall(name='recall'),  # mide falsos negativos
            metrics.F1Score(name='f1_score', average='macro')  # media F1 por clase
        ])
    return modelo

modelo_bn = batchnorm()
modelo_bn.summary()

In [ ]:
hist_bn, loss_bn, acc_bn, prec_bn, rec_bn, f1_bn = entrenar(
    modelo_bn,
    conjunto_entrenamiento,
    conjunto_validacion,
    conjunto_test)

Epoch 1/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 95ms/step - accuracy: 0.4735 - f1_score: 0.4708 - loss: 1.4696 - precision: 0.6777 - recall: 0.2603 - val_accuracy: 0.3440 - val_f1_score: 0.3258 - val_loss: 2.0071 - val_precision: 0.4642 - val_recall: 0.2400
Epoch 2/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 98ms/step - accuracy: 0.5130 - f1_score: 0.5109 - loss: 1.4108 - precision: 0.7031 - recall: 0.2907 - val_accuracy: 0.3720 - val_f1_score: 0.3359 - val_loss: 1.9003 - val_precision: 0.4619 - val_recall: 0.2850
Epoch 3/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 97ms/step - accuracy: 0.5357 - f1_score: 0.5326 - loss: 1.3234 - precision: 0.7184 - recall: 0.3322 - val_accuracy: 0.3140 - val_f1_score: 0.2401 - val_loss: 2.2819 - val_precision: 0.3700 - val_recall: 0.2490
Epoch 4/15
125/125 ━━━━━━━━━━━━━━━━━━━━ 13s 99ms/step - accuracy: 0.5683 - f1_score: 0.5643 - loss: 1.2437 - precision: 0.7323 - recall: 0.3898 - val_accuracy: 0.4000 - val_f1_score: 0.3546 - val_loss: 1.8584 - val_precision: 0.5385 - val

#### 3.6) Modelo 6 - Híbrido

### 4. Comparación y resumen de resultados

In [ ]:
import pandas as pd

# Creamos la tabla con los resultados obtenidos
# (sustituye los valores entre corchetes por tus variables reales si cambian los nombres)

results = pd.DataFrame({
    "Modelo": ["Vanilla","Dropout","L1","L2 ","BatchNorm"],
    "Regularización": ["Ninguna","Dropout(x)","L1(x)","L2(x)","Batch Normalization"],    # pongo (x) para poner luego los parámetros buenos
    "Accuracy": [acc_vanilla * 100,acc_dropout * 100,acc_L1 * 100,acc_L2 * 100,acc_bn * 100],
    "Precision": [prec_vanilla,prec_dropout,prec_L1,rec_L2,prec_bn],
    "Recall": [rec_vanilla,rec_dropout,rec_L1,rec_L2,rec_bn],
    "F1-Score": [f1_vanilla,f1_dropout,f1_L1,f1_L2,f1_bn]
})

results = results.round({"Accuracy": 3,"Precision": 3,"Recall": 3,"F1-Score": 3})

# Mostramos la tabla ordenada por Accuracy descendente
results.sort_values(by="Accuracy", ascending=False).reset_index(drop=True)

,Modelo,Regularización,Accuracy,Precision,Recall,F1-Score
0,L2,L2(x),38.050,0.228,0.228,0.367
1,Vanilla,Ninguna,34.300,0.515,0.206,0.327
2,L1,L1(x),31.137,0.591,0.107,0.279
3,BatchNorm,Batch Normalization,19.275,0.199,0.182,0.148
4,Dropout,Dropout(x),18.050,0.000,0.000,0.111


### 6. Conclusión

La red neuronal base aprende rápido y obtiene una precisión aceptable en entrenamiento, pero empieza a sobreajustar. Esto se ve porque la accuracy de entrenamiento sigue subiendo mientras que la accuracy de validación deja de mejorar e incluso baja ligeramente. Eso significa que el modelo se está adaptando demasiado a las imágenes concretas del set de entrenamiento y no generaliza tan bien a imágenes nuevas.

Cuando añadimos regularización (L2 y Dropout) el entrenamiento es más lento y la accuracy de entrenamiento es un poco más baja, pero la accuracy de validación es más estable y en general más cercana a la de entrenamiento. Esto indica que el modelo es menos dependiente de detalles concretos del conjunto de entrenamiento. En el conjunto de test, que es el que no usamos nunca para ajustar nada, el modelo regularizado mejora ligeramente el resultado respecto al modelo base. Esto sugiere que la regularización ayuda a la red a generalizar mejor.

Como desventaja, el modelo regularizado tarda un poco más en converger y necesita más épocas para exprimir su potencial, porque al apagar neuronas (Dropout) y penalizar pesos grandes (L2) le cuesta un poco más “memorizar” patrones. Aun así, para este problema de clasificación de imágenes, es preferible un modelo que generaliza mejor aunque entrene un poco más lento.